In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/statoil-iceberg-classifier-challenge/train.json.7z
/kaggle/input/statoil-iceberg-classifier-challenge/test.json.7z
/kaggle/input/statoil-iceberg-classifier-challenge/sample_submission.csv.7z


In [2]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

statoil-iceberg-classifier-challenge



In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [4]:
import pandas as pd
!pip install py7zr
import py7zr
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import Sequential
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import SGD,Adam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.5/390.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.6/138.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.0 MB/s eta 0:00:00


In [5]:
with py7zr.SevenZipFile('/kaggle/input/statoil-iceberg-classifier-challenge/train.json.7z', mode='r') as z:
    z.extractall()
    
with py7zr.SevenZipFile('/kaggle/input/statoil-iceberg-classifier-challenge/test.json.7z', mode='r') as z1:
    z1.extractall()

In [6]:
#Load data
train = pd.read_json('/kaggle/working/data/processed/train.json')
test = pd.read_json('/kaggle/working/data/processed/test.json')
train.head()

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
2,58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [7]:
train

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
2,58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0
...,...,...,...,...,...
1599,04e11240,"[-30.999878, -29.976866, -28.233906, -29.50732...","[-27.847719, -28.233864, -24.712077999999998, ...",na,0
1600,c7d6f6f8,"[-25.31155, -26.511555, -28.694487, -27.180115...","[-29.563713, -28.290375, -26.839405, -28.29046...",na,0
1601,bba1a0f1,"[-18.141895, -18.141844, -19.01737, -19.701599...","[-25.305355, -29.387701, -28.963863, -26.16023...",na,0
1602,7f66bb44,"[-22.455633, -25.794661, -26.954567, -22.83354...","[-26.070356, -22.093737, -21.577662, -24.53376...",na,0


## Intro about the Data
Sentinel-1 위성은 지구로부터 약 680km 떨어진 곳에 위치하고 있습니다. 특정 입사각에서 신호를 발사하고 반사된 신호를 기록합니다. 이 반사된 신호를 백스캐터(backscatter)라고 합니다. 데이터는 백스캐터 계수로, 다음과 같은 관계로 표현됩니다:

$σo (dB) = βo (dB) + 10log10 [ sin(ip) / sin (ic)] $

여기서,

ip는 특정 픽셀의 입사각
ic는 이미지 중심의 입사각
K는 상수
우리는 데이터로 직접적으로 주어진 $σo$를 가지고 있습니다.

이제 $σo$의 특징에 대해 살펴보겠습니다.
기본적으로 $σo$는 신호가 반사된 표면에 따라 다양하게 변합니다. 예를 들어, 특정 입사각에서 다음과 같이 변합니다:

물........... 건물........ 농업........... 불모지........


1.HH: -27.001 ................ 2.70252 ................. -12.7952 ................ -17.25790909

2.HV: -28.035 ................ -20.2665 .................. -21.4471 ................. -20.019

보시다시피, HH 성분은 매우 다양하게 변하지만 HV는 그렇지 않습니다. ship scatter 데이터는 가지고 있지 않지만, 금속 물체이므로 빙하와는 다른 방식으로 변할 것으로 예상됩니다.

### What is HH HV?
이해를 돕기 위해, Sentinel-1 위성은 RISTSAT(인도 원격 감지 위성)과 동등하며, H 편광만을 사용하여 신호를 발사합니다. 이 H-편광 신호가 산란되고, 물체가 편광이 변경되어 H와 V의 혼합으로 반환됩니다. 따라서 Sentinel은 H-편광 전송기만을 갖고 있기 때문에 반환 신호는 HH와 HV 형태만 가지게 됩니다. VV가 주어지지 않은 이유는 Sentinel에는 V-편광 신호 발사기가 없기 때문입니다.

이제 특징에 대해 더 자세히 알아보기 위해 두 개의 밴드에서 평균을 내어 3채널 RGB와 동등한 형태의 3채널 데이터를 생성합니다.

(+머신비전 참고)


## 위성으로 찍은 사진을 기반으로 그 물체가 빙하인지 배인지 분류하는 대회

*  "band_1"은 Sentinel-1 위성이 보낸 백스캐터 데이터
(백스캐터란 위성이 지구 표면에 보낸 신호가 대상물체(지표)에 부딪혀 반사되어 다시 위성으로 돌아오는 데이터)

In [8]:
#Generate the training data
# "train" 데이터의 "band_1" 컬럼에서 각각의 값을 "band" 변수로 가져옵니다.
# "band" 값을 numpy 배열로 변환합니다.
# "astype(np.float32)"를 사용하여 배열의 데이터 타입을 float32로 변환합니다.
#"reshape(75, 75)"를 사용하여 배열의 크기를 75x75로 재구성합니다.
# 위의 단계를 "band_1" 컬럼의 모든 값에 대해 반복하여, 각각의 값을 numpy 배열로 변환하고 float32 자료형으로 변환하며, 75x75 크기의 배열로 재구성합니다.
# 최종적으로, "X_band_1" 변수에는 "train" 데이터의 "band_1" 컬럼에 있는 값들이 numpy 배열로 저장되어 있게 됩니다.

X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_1

array([[[-27.87836 , -27.15416 , -28.668615, ..., -25.573483,
         -26.488674, -30.507013],
        [-28.66853 , -27.8784  , -27.508776, ..., -26.488632,
         -28.671562, -31.594166],
        [-28.66853 , -27.15416 , -25.865042, ..., -27.157106,
         -27.881393, -32.837124],
        ...,
        [-29.092905, -28.669163, -30.504612, ..., -30.011005,
         -25.868538, -26.489265],
        [-31.591682, -27.878992, -28.669205, ..., -27.512272,
         -26.817074, -27.512358],
        [-29.092905, -26.814081, -27.15479 , ..., -26.817032,
         -27.881983, -28.268127]],

       [[-12.242375, -14.920305, -14.920363, ..., -10.633402,
         -11.956054, -14.379026],
        [-13.046626, -14.077841, -12.728542, ..., -11.243189,
         -11.786291, -14.379026],
        [-15.248114, -14.0779  , -12.604514, ..., -15.680162,
         -13.51685 , -14.454887],
        ...,
        [-15.589596, -16.899685, -17.527912, ..., -11.78711 ,
         -10.883234, -12.306804],
        [-14

In [9]:
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

In [10]:
#"X_band_1" 배열에는 "train" 데이터의 "band_1" 컬럼의 값들이 75x75x1 형태의 3차원 배열로 저장되어 있습니다.
#"X_band_2" 배열에는 "train" 데이터의 "band_2" 컬럼의 값들이 75x75x1 형태의 3차원 배열로 저장되어 있습니다.
#"X_band_1"과 "X_band_2" 배열에 각각 "np.newaxis"를 사용하여 새로운 축을 추가합니다. 
# 이를 통해 각 배열은 4차원 배열로 변환됩니다.
#"((X_band_1+X_band_2)/2)"를 사용하여 "X_band_1"과 "X_band_2" 배열의 평균값을 계산
# 이를 다시 75x75x1 형태의 3차원 배열로 저장합니다.
# "np.concatenate" 함수를 사용하여 "X_band_1", "X_band_2", 그리고 평균값 배열을 axis=-1 방향(마지막 축)으로 결합
# 최종적으로, "X_train" 변수에는 "X_band_1", "X_band_2", 그리고 두 배열의 평균값이 4차원 배열로 저장
# 이는 RGB 형태의 3개의 채널을 가진 이미지 데이터로 사용될 수 있습니다.
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [11]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
def plotmy3d(c, name):

    data = [
        go.Surface(   # go.Surface: pyplot에 정의된 객체 이름. 3D 플롯 데이터 설정
            z=c  # z: 표면의 높이값
        )
    ]
    layout = go.Layout(   # go.Layout: Plotly 라이브러리에서 사용되는 그래프 레이아웃을 정의하는 객체
        title=name,
        autosize=False,   # 자동 크기 조정 비활성화. 지정한 폭, 넓이로 설정될 것
        width=700,
        height=700,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)
plotmy3d(X_band_1[12,:,:], 'iceberg')

레이더 데이터는 실제 이미지가 아닌 산점도 형태로 표현되기 때문에 `빙산`의 형태는 `산의 모양`과 유사하게 나타납니다. 빙산의 형태에는 뾰족한 부분이나 왜곡된 부분이 있을 수 있습니다. 반면에 `선박`은 `점 또는 길쭉한 점`과 같은 형태로 나타날 것입니다. 이러한 빙산과 선박의 형태 차이를 이용하여 합성 이미지(composite images)를 생성하면, 레이더 데이터를 활용하여 구조적 차이를 감지하고 이를 학습하는 합성곱 신경망(CNN) 등의 머신러닝 알고리즘을 활용하여 빙산과 선박을 구분하는데 도움이 될 수 있습니다.




In [12]:
plotmy3d(X_band_1[14,:,:], 'Ship')

In [13]:
plotmy3d(X_band_1[11,:,:], 'test')

이미지에서 선박은 길쭉한 점과 같은 형태로 보입니다. 


+) 시각화할 만한 이미지 해상도가 별로 없음. 그러나 CNN은 이런 것을 도와줌 => keras를 사용한 CNN으로 구축







In [14]:
#Import Keras
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import BatchNormalization  # 수정
from tensorflow.keras.layers import concatenate   # 수정
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [15]:
# define our model
def get_model():
    #Building the model
    gmodel = Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(75, 75,3)))
    gmodel.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    gmodel.add(Dropout(0.2))
    
    #Conv Layer 2
    gmodel.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    gmodel.add(Dropout(0.2))
    
    #Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    gmodel.add(Dropout(0.2))
    
    #Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    gmodel.add(Dropout(0.2))

    #Flatten the data for upcoming dense layers : dense layers 평면화. 다차원의 데이터를 1차원 벡터로 변환
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))   #과적합 방지

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel

In [16]:
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)